In [42]:
import numpy as np
import matplotlib.pyplot as plt
import os
from ikrlib import *


if not os.path.isfile("SUR_projekt2022-2023.zip"):
    !wget https://www.fit.vutbr.cz/study/courses/SUR/public/projekt_2022-2023/SUR_projekt2022-2023.zip
    !unzip SUR_projekt2022-2023.zip


In [43]:
CLASSES = 2

train_x = np.empty((0,80,80))
train_y = np.empty((0),dtype=int)

# TODO pridat vse nechci si uvarit PC
# TODO mozna bych zkonkatenoval mfcc do train_x
for i in range(1,CLASSES+1): 
    train_i = np.array(list(png2fea('train/'+str(i)).values()))
    label_i = np.full(len(train_i),i)
    #add to train_x
    train_x = np.concatenate((train_x, train_i), axis=0)
    train_y = np.concatenate((train_y,label_i),axis=0)


print(train_x.shape, train_y.shape)
sample1 = train_x[0]
print(train_x.shape)

Processing file:  train/1/f401_01_f18_i0_0.png
Processing file:  train/1/f401_01_r09_i0_0.png
Processing file:  train/1/f401_02_f18_i0_0.png
Processing file:  train/1/f401_02_r09_i0_0.png
Processing file:  train/1/f401_03_f18_i0_0.png
Processing file:  train/1/f401_03_r09_i0_0.png
Processing file:  train/2/f402_01_f18_i0_0.png
Processing file:  train/2/f402_01_r09_i0_0.png
Processing file:  train/2/f402_02_f18_i0_0.png
Processing file:  train/2/f402_02_r09_i0_0.png
Processing file:  train/2/f402_03_f18_i0_0.png
Processing file:  train/2/f402_03_r09_i0_0.png
(12, 80, 80) (12,)
(12, 80, 80)


In [55]:
flat = train_x.reshape(train_x.shape[0],-1)
print(flat.shape)


(12, 6400)


In [59]:
#PCA 
cov = np.cov(flat.T)

eig_vals, eig_vecs = np.linalg.eig(cov)

# sort
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# print first 10
for i in range(10):
    print(eig_pairs[i])



(2688866.1751426966, array([-0.00292707+0.j, -0.00410764+0.j, -0.00341273+0.j, ...,
       -0.01422124+0.j, -0.01291913+0.j, -0.01029904+0.j]))
(1204011.4282300281, array([-0.01704999+0.j, -0.01930412+0.j, -0.01938977+0.j, ...,
        0.00312058+0.j,  0.00211808+0.j, -0.00087295+0.j]))
(970328.3915503409, array([0.01061933+0.j, 0.01081768+0.j, 0.01194694+0.j, ...,
       0.00500877+0.j, 0.00422902+0.j, 0.0059948 +0.j]))
(931417.1447661449, array([0.00891695+0.j, 0.01107032+0.j, 0.01516769+0.j, ...,
       0.00217012+0.j, 0.00347769+0.j, 0.00371411+0.j]))
(819316.4573963934, array([-0.02430097+0.j, -0.02172785+0.j, -0.01999234+0.j, ...,
       -0.01600654+0.j, -0.018103  +0.j, -0.01443782+0.j]))
(578077.0664830212, array([-0.0380795 +0.j, -0.0360707 +0.j, -0.03187474+0.j, ...,
       -0.00033318+0.j, -0.00186124+0.j, -0.00339203+0.j]))
(525333.8154352703, array([-0.00258351+0.j,  0.00421105+0.j, -0.00312384+0.j, ...,
       -0.020855  +0.j, -0.01960709+0.j, -0.01275844+0.j]))
(341787.1

In [122]:
# convert to real with real_if_close
eig_pairs = [(np.real_if_close(eig_pairs[i][0]), np.real(eig_pairs[i][1])) for i in range(len(eig_pairs))]

# print first 10 vectors
for i in range(20):
    print(eig_pairs[i][1])
    
# transform to first 50 eigenvectors
train_x_pca = np.array([np.dot(eig_pairs[i][1], flat.T) for i in range(50)]).T
print(train_x_pca.shape)

[-0.00292707 -0.00410764 -0.00341273 ... -0.01422124 -0.01291913
 -0.01029904]
[-0.01704999 -0.01930412 -0.01938977 ...  0.00312058  0.00211808
 -0.00087295]
[0.01061933 0.01081768 0.01194694 ... 0.00500877 0.00422902 0.0059948 ]
[0.00891695 0.01107032 0.01516769 ... 0.00217012 0.00347769 0.00371411]
[-0.02430097 -0.02172785 -0.01999234 ... -0.01600654 -0.018103
 -0.01443782]
[-0.0380795  -0.0360707  -0.03187474 ... -0.00033318 -0.00186124
 -0.00339203]
[-0.00258351  0.00421105 -0.00312384 ... -0.020855   -0.01960709
 -0.01275844]
[-0.00691099 -0.01051105 -0.0143024  ...  0.01308498  0.00974018
 -0.00224841]
[-0.00010406  0.00322321  0.0052832  ...  0.0061945   0.00499106
  0.00151761]
[ 0.01892485  0.02240387  0.02726241 ... -0.00280057 -0.00550144
 -0.01113927]
[ 0.01288447  0.01414558  0.00942853 ...  0.00975032  0.00561955
 -0.00566277]
[-0.3147417  -0.00833048  0.00146708 ...  0.00768816  0.0005667
  0.00677945]
[ 0.00083055 -0.00430509  0.01202593 ...  0.00316047  0.00085092
 -0.

In [129]:
#train GMM
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=CLASSES, covariance_type='full', max_iter=1000, random_state=0)
gmm.means_ = np.array([train_x_pca[train_y == i].mean(axis=0) for i in range(1,CLASSES+1)])
gmm.fit(train_x_pca)


GaussianMixture(max_iter=1000, n_components=2, random_state=0)

In [130]:
#train data accuarcy
predict_y = gmm.predict(train_x_pca) + 1
print(predict_y)
print(train_y)

print("Train data accuarcy: ", np.sum(predict_y == train_y)/len(train_y))

[2 1 1 1 1 1 2 2 2 2 2 2]
[1 1 1 1 1 1 2 2 2 2 2 2]
Train data accuarcy:  0.9166666666666666


In [131]:
#create Test data
test_x = np.empty((0,80,80))
test_y = np.empty((0),dtype=int)

for i in range(1,CLASSES+1): 
    test_i = np.array(list(png2fea('dev/'+str(i)).values()))
    label_i = np.full(len(test_i),i)
    #add to test_x
    test_x = np.concatenate((test_x, test_i), axis=0)
    test_y = np.concatenate((test_y,label_i),axis=0)
print(test_x.shape, test_y.shape)

# flatten test data
test_x = test_x.reshape(test_x.shape[0],-1)

# transform to first 50 eigenvectors
test_x_pca = np.array([np.dot(eig_pairs[i][1], test_x.T) for i in range(50)]).T
print(test_x_pca.shape)


Processing file:  dev/1/f401_04_f18_i0_0.png
Processing file:  dev/1/f401_04_r09_i0_0.png
Processing file:  dev/2/f402_04_f18_i0_0.png
Processing file:  dev/2/f402_04_r09_i0_0.png
(4, 80, 80) (4,)
(4, 50)


In [132]:
#test data accuarcy
predict_y = gmm.predict(test_x_pca) + 1
print(predict_y)
print(test_y)
print("Test data accuarcy: ", np.sum(predict_y == test_y)/len(test_y))


[1 2 2 2]
[1 1 2 2]
Test data accuarcy:  0.75
